### Purpose of this notebook is to view the previous results and see how this MrSQL Fitting issue impacts results

In [50]:
#Sktime imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.shapelet_based import MrSEQLClassifier
from sklearn.model_selection import GroupShuffleSplit

from IPython.utils import io
import pandas as pd
import glob
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
import sys
from os.path import exists

import importlib

# EEG package modules__________________________________________________________________
import constants
import custom_ts_length
# import eeg_stat_ts
import format_eeg_data
import run_expts

import calculate_results

In [51]:
core_path = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/'

#1 - This is to run the custom length expts
folders = {'N1': 'N1_customised_40_expt_files/' , 'N2' : 'N2_120_expt_files_v2/','N3' : 'N3_82_expt_files/', 'REM' :'REM_19_expt_files/'}

# #2 -This is to run the expts on the full original length data without any dropping or cutting of the TS 
# folders = {'N1': 'N1_full_expts/' , 'N2' : 'N2_full_expts/','N3' : 'N3_full_expts/', 'REM' :'REM_full_expts/'}

# #3 - This is to run the wakeful eyes closed experiments
# folders = {'EC1': 'EC_8_expts/' , 'EC2' : 'EC_full_expts/' }

In [52]:
folder_names = [val for val in folders.values()]
print(folder_names)
folder = folder_names[1]
load_path = core_path + folder
print(load_path)

['N1_customised_40_expt_files/', 'N2_120_expt_files_v2/', 'N3_82_expt_files/', 'REM_19_expt_files/']
/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/N2_120_expt_files_v2/


In [37]:
#pd.read_hdf(load_path + 'y_full.h5', key='df', mode='r') 

In [53]:
folder = load_path
sub_fol_an_file = [file[len(folder):] for file in glob.glob(folder+'*')]
sub_fol_an_file

['groups.h5',
 'X.h5',
 'y_full.h5',
 'expt_1',
 'expt_2',
 'expt_3',
 'expt_4',
 'MrSQL_expt_1',
 'MrSQL_expt_2',
 'MrSQL_expt_3',
 'MrSQL_expt_4',
 'MrSQL_SAX_expt_1',
 'MrSQL_SAX_expt_2']

In [54]:
load_path

'/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/N2_120_expt_files_v2/'

In [58]:
folder = load_path + 'MrSQL_expt_1/'
sub_fol_an_file = [file[len(folder):] for file in glob.glob(folder+'*')]
sub_fol_an_file

['int_df.h5',
 'raw_clfs_results.h5',
 'regional_clf_acc_std_results.h5',
 ' raw_ensemble_predictions.h5',
 'processed_results_full_ensemble_df.h5']

In [59]:
pd.read_hdf(folder + 'regional_clf_acc_std_results.h5', key='df', mode='r') 

,random_state,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_1_std,Prefrontal_MrSQL_2_acc,Prefrontal_MrSQL_2_std,Prefrontal_MrSQL_3_acc,Prefrontal_MrSQL_3_std,Prefrontal_MrSQL_4_acc,Prefrontal_MrSQL_4_std,Frontal_MrSQL_1_acc,...,Right Parietal_MrSQL_4_acc,Right Parietal_MrSQL_4_std,Occipital_MrSQL_1_acc,Occipital_MrSQL_1_std,Occipital_MrSQL_2_acc,Occipital_MrSQL_2_std,Occipital_MrSQL_3_acc,Occipital_MrSQL_3_std,Occipital_MrSQL_4_acc,Occipital_MrSQL_4_std
0,13,0.904762,0.131277,0.933333,0.133333,0.933333,0.133333,0.860317,0.162929,0.765079,...,0.871429,0.123810,0.833333,0.164804,0.826984,0.108635,0.826984,0.108635,0.849206,0.107656
1,37,0.949206,0.063014,1.000000,0.000000,1.000000,0.000000,0.949206,0.063014,0.720635,...,0.871111,0.175766,0.725079,0.268704,0.804444,0.301240,0.826667,0.258543,0.826667,0.258543


### Investigate per fold performances

In [60]:
calculate_results.calc_per_fold_metrics(pd.read_hdf(folder + 'raw_clfs_results.h5', key='df', mode='r') , expt_type = 'binary' )

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

,y_true,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_1_prec,Prefrontal_MrSQL_1_recall,Prefrontal_MrSQL_1_y_preds,Prefrontal_MrSQL_2_acc,Prefrontal_MrSQL_2_prec,Prefrontal_MrSQL_2_recall,Prefrontal_MrSQL_2_y_preds,Prefrontal_MrSQL_3_acc,...,Occipital_MrSQL_2_recall,Occipital_MrSQL_2_y_preds,Occipital_MrSQL_3_acc,Occipital_MrSQL_3_prec,Occipital_MrSQL_3_recall,Occipital_MrSQL_3_y_preds,Occipital_MrSQL_4_acc,Occipital_MrSQL_4_prec,Occipital_MrSQL_4_recall,Occipital_MrSQL_4_y_preds
0,"[0, 0, 0, 0, 0, 0, 1, 1, 1]",0.666667,0.0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.666667,0.0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.666667,...,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.666667,0.0,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.666667,0.0,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.0,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.0,"[0, 0, 0, 0, 0, 0]",1.000000,...,0.0,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.000000,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.000000,"[0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 0, 1]",0.857143,0.5,1.0,"[0, 0, 0, 1, 0, 0, 1]",1.000000,1.0,1.0,"[0, 0, 0, 0, 0, 0, 1]",1.000000,...,0.0,"[0, 0, 0, 0, 0, 0, 0]",0.857143,0.0,0.000000,"[0, 0, 0, 0, 0, 0, 0]",0.857143,0.0,0.000000,"[0, 0, 0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0, 0, 0, 1, 1]",1.000000,1.0,1.0,"[0, 0, 0, 0, 0, 0, 0, 1, 1]",1.000000,1.0,1.0,"[0, 0, 0, 0, 0, 0, 0, 1, 1]",1.000000,...,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.777778,0.0,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.888889,1.0,0.500000,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,"[0, 0, 0, 0, 1, 1]",1.000000,1.0,1.0,"[0, 0, 0, 0, 1, 1]",1.000000,1.0,1.0,"[0, 0, 0, 0, 1, 1]",1.000000,...,0.5,"[0, 0, 0, 0, 1, 0]",0.833333,1.0,0.500000,"[0, 0, 0, 0, 1, 0]",0.833333,1.0,0.500000,"[0, 0, 0, 0, 1, 0]"
5,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.0,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.0,"[0, 0, 0, 0, 0, 0]",1.000000,...,0.0,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.000000,"[0, 0, 0, 0, 0, 0]",1.000000,0.0,0.000000,"[0, 0, 0, 0, 0, 0]"
6,"[0, 0, 0, 1, 1]",1.000000,1.0,1.0,"[0, 0, 0, 1, 1]",1.000000,1.0,1.0,"[0, 0, 0, 1, 1]",1.000000,...,0.5,"[0, 0, 0, 1, 0]",0.800000,1.0,0.500000,"[0, 0, 0, 1, 0]",0.800000,1.0,0.500000,"[0, 0, 0, 1, 0]"
7,"[0, 0, 0, 0, 0, 0, 1]",0.857143,0.5,1.0,"[0, 0, 0, 0, 1, 0, 1]",1.000000,1.0,1.0,"[0, 0, 0, 0, 0, 0, 1]",1.000000,...,1.0,"[0, 0, 0, 0, 0, 0, 1]",1.000000,1.0,1.000000,"[0, 0, 0, 0, 0, 0, 1]",1.000000,1.0,1.000000,"[0, 0, 0, 0, 0, 0, 1]"
8,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",0.888889,0.5,1.0,"[0, 0, 0, 0, 0, 0, 1, 0, 1]",1.000000,1.0,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1.000000,...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1.000000,1.0,1.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1.000000,1.0,1.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
9,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",1.000000,1.0,1.0,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",1.000000,1.0,1.0,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",1.000000,...,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.333333,1.0,0.142857,"[0, 0, 0, 0, 1, 0, 0, 0, 0]",0.333333,1.0,0.142857,"[0, 0, 0, 0, 0, 0, 1, 0, 0]"
